In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


import datetime

import re
from openpyxl import Workbook, load_workbook

In [8]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [13]:
HOME_URL = 'http://kcna.kp/kp'
RECENT_URL = 'http://kcna.kp/kp/category/articles/q/1ee9bdb7186944f765208f34ecfb5407.kcmsf'
browser.get(HOME_URL)
browser.get(RECENT_URL)

In [15]:
article_wrapper = browser.find_element(by=By.CLASS_NAME, value='article-link')

In [16]:
a_tag = article_wrapper.find_element(by=By.TAG_NAME, value='a')

In [17]:
a_tag.click()

In [72]:
def get_detail_info(browser):
    title = browser.find_element(by=By.CLASS_NAME, value='article-main-title').text
    content_wrapper = browser.find_element(by=By.CLASS_NAME, value='content-wrapper')
    p_tags = content_wrapper.find_elements(by=By.TAG_NAME, value='p')
    
    pre_content1 = ''
    pre_content2 = ''
    pre_content3 = ''
    content = ''
    for p_tag in p_tags:
        text = p_tag.text
        if text.startswith('(') and text.endswith(')'):
            text = text.replace("(","").replace(")","")
            split_text = text.split(' ')
            pre_content1 = split_text[0]
            pre_content2 = split_text[1] + ' ' + split_text[2]
            pre_content3 = split_text[3]
        else :
            content += text + '\r\n'
    
    if len(content) >= 32000:
        content = content[0:31980] + '\r\n (이하 절단)'
    
    return title, pre_content1, pre_content2, pre_content3, content

In [81]:
get_detail_info(browser)

('주체111(2022)년 7월 28일 신문개관',
 '평양',
 '7월 28일발',
 '조선중앙통신',
 '28일 중앙신문들에 실린 주요소식과 글들\r\n-위대한 전승 69돐 기념행사가 성대히 진행된 소식과 사진문헌들\r\n-경애하는 김정은동지께서 위대한 전승 69돐 기념행사에서 하신 연설(사진문헌들을 함께 편집)\r\n-경애하는 김정은동지께서 전승절에 즈음하여 조국해방전쟁참전렬사묘를 찾으신 소식과 사진문헌들\r\n-위대한 수령 김일성동지와 위대한 령도자 김정일동지의 동상에 일군들과 근로자들,인민군장병들,청소년학생들 꽃바구니 진정\r\n-만수대언덕에 높이 모신 위대한 수령 김일성동지와 위대한 령도자 김정일동지의 동상에 김일성김정일기금리사회 명예부리사장이 꽃바구니를 보내여왔다.\r\n-전승절에 즈음하여 조선인민군 장병들 금수산태양궁전을 찾아 숭고한 경의 표시\r\n-제8차 전국로병대회 참가자들이 당의 품속에서 영생하는 옛 지휘관들과 전우들을 찾아 경의를 표시하였다.\r\n-각지 일군들과 근로자들,청년학생들 전승절을 뜻깊게 맞이\r\n-청년학생들의 야회 진행\r\n-전승컵체육경기대회,전승컵 전국태권도강자경기대회,전승컵보링경기대회 진행\r\n-제8차 전국로병대회에 참가한 로병들의 군공을 소개\r\n-전국적인 전염병전파 및 치료상황 통보\r\n-전망적인 전염병위협을 예견한 물리적,기술적대책 실행\r\n-6.25-7.27반미공동투쟁월간에 즈음하여 여러 나라에서 행사 진행\r\n-세계적인 대류행전염병전파실태(끝)\r\n')

In [48]:
def trans_year(north_year):
    return 1911 + int(north_year)

In [64]:
start_date = datetime.date(2022, 7, 27)
end_date = datetime.date(2022, 7, 27)

In [92]:
global rownum
rownum = 0

In [88]:
def write_excel(append_row):
    wb = load_workbook('./output/조선중앙통신.xlsx', data_only=True)
    ws = wb.active
    ws.append(append_row)
    wb.save('./output/조선중앙통신.xlsx')

In [95]:
def crawl_list_page(browser):
    article_link = browser.find_element(by=By.CLASS_NAME, value='article-link')
    li_tags = article_link.find_elements(by=By.TAG_NAME, value='li')
    for li_tag in li_tags:
        isTarget, isGo, targetUrl, pub_date = check_target(li_tag, start_date, end_date)
        if isTarget :
            browser.switch_to.new_window('tab')
            browser.get(targetUrl)
            title, pre_content1, pre_content2, pre_content3, content = get_detail_info(browser)
            global rownum
            rownum += 1
            write_excel([rownum, pub_date, title, pre_content1, pre_content2, pre_content3, content])
            browser.close() 
            browser.switch_to.window(browser.window_handles[0])
    
    return isGo

In [97]:
isGo = True
while(isGo):
    isGo = crawl_list_page(browser)    
    go_next_page(browser)

In [76]:
title, pre_content1, pre_content2, pre_content3, content

('조국해방전쟁을 승리에로 이끄신 위대한 수령 김일성동지',
 '평양',
 '7월 27일발',
 '조선중앙통신',
 '위대한 수령 김일성동지께서는 조국해방전쟁의 모든 중하를 한몸에 지니시고 주체적인 군사사상과 전법,탁월한 령도로 영웅조선의 백승의 력사와 전통을 창조하시였으며 조국과 민족의 존엄과 영예를 온 세상에 높이 떨쳐주시였다.(끝)\r\n')

In [82]:
def check_target(li_tag, start_date, end_date):
    detail_url = li_tag.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
    publish_date = li_tag.find_element(by=By.CLASS_NAME, value='publish-time')
   
    date = publish_date.text.replace("[","").replace("]","").replace("주체","").split(".")
    year = int(trans_year(date[0]))
    month = int(date[1])
    day = int(date[2])

    pub_date = datetime.date(year, month, day)
    if start_date <= pub_date and pub_date <= end_date :
        isTarget = True
        isGo = True
    elif pub_date < start_date :
        isTarget = False
        isGo = False
    elif pub_date > end_date :
        isTarget = False
        isGo = True
    return isTarget, isGo, detail_url, pub_date

In [52]:
article_links = browser.find_elements(by=By.CLASS_NAME, value='article-link')
for article_link in article_links:
    isTarget, isGo = check_target(article_link, start_date, end_date)
    print(isTarget, isGo)

no target - 종료 2022-07-23
False False


In [39]:
def go_next_page(browser):
    next_btn = browser.find_element(by=By.CLASS_NAME, value='next-ctrl').find_element(by=By.TAG_NAME, value='a')
    next_btn.click()

In [45]:
go_next_page(browser)

In [ ]:
browser.switch_to.new_window('tab')
browser.close()

In [79]:
# 엑셀 생성
wb = Workbook()
ws = wb.active

# 제목 적기
sub = ['번호','게재일자','기사 제목','(내용1)','내용2','내용3','기사 내용']
for kwd, j in zip(sub, list(range(1, len(sub)+1))):
    ws.cell(row=1, column=j).value = kwd

wb.save('./output/조선중앙통신.xlsx')

In [96]:
# 팝업창 종료 후 close
browser.close() 
browser.switch_to.window(browser.window_handles[0])